<a href="https://www.kaggle.com/code/tiheli/mae-mse-mape-with-insurance?scriptVersionId=207964850" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance1/insurance.csv


In [2]:
# Loading the dataset from a CSV file
df = pd.read_csv('/kaggle/input/insurance1/insurance.csv')

In [3]:
# Importing necessary libraries for regression and data splitting
from sklearn.linear_model import LinearRegression  # Linear Regression model for predicting continuous outcomes
from sklearn.model_selection import train_test_split  # Function to split data into training and test sets
# Importing metrics for evaluating model performance
from sklearn.metrics import mean_squared_error  # To calculate the Mean Squared Error (MSE)
from sklearn.metrics import mean_absolute_error  # To calculate the Mean Absolute Error (MAE)
from sklearn.metrics import mean_absolute_percentage_error  # To calculate the Mean Absolute Percentage Error (MAPE)

In [4]:
# Displaying the first few rows of the DataFrame to preview the dataset's structure and initial data
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
# Converting categorical columns into dummy/indicator variables for model compatibility
# 'drop_first=True' avoids multicollinearity by dropping the first category in each column
df = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)

In [6]:
# Displaying the first few rows of the DataFrame to preview the dataset's structure and initial data
df.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,False,True,False,False,True
1,18,33.770,1,1725.55230,True,False,False,True,False
2,28,33.000,3,4449.46200,True,False,False,True,False
3,33,22.705,0,21984.47061,True,False,True,False,False
4,32,28.880,0,3866.85520,True,False,True,False,False


In [7]:
# Defining the target variable 'y' as the 'charges' column, representing the insurance charges to predict
y = df[['charges']]

# Defining the feature set 'x' by dropping the target column 'charges' from the DataFrame
x = df.drop("charges", axis=1)

In [8]:
# Splitting the data into training and testing sets
# 80% of the data is used for training, and 20% for testing
# Setting random_state=42 for consistent and reproducible results
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.80, random_state=42)

In [9]:
# Initializing the Linear Regression model
lm = LinearRegression()

# Fitting the model to the training data to learn the relationship between features and target
model = lm.fit(x_train, y_train)

# Calculating and displaying the R-squared score of the model on the test set
# This score indicates how well the model explains the variance in the test data
model.score(x_test, y_test)

0.7835929767120724

In [10]:
# Making a prediction using a new sample with specific feature values
# Wrapping the sample in a list to match the expected input shape for prediction
model.predict([[19, 26, 0, 1, 1, 0, 0, 1]])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[24538.46352107]])

In [11]:
# Creating an empty DataFrame to store potential mistakes or errors in the data
# This can be used for analysis or debugging purposes later
df_ms = pd.DataFrame()  # ms = mistake

In [12]:
# Adding the target variable 'y' (insurance charges) to the mistakes DataFrame for reference
# This allows for easier comparison and analysis of predicted vs. actual values later
df_ms['y'] = y

In [13]:
# Displaying the mistakes DataFrame to review its contents and check the added target variable 'y'
df_ms

,y
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [14]:
# Making predictions on the entire feature set 'x' to obtain predicted insurance charges
y_predict = model.predict(x)

In [15]:
# Adding the predicted values to the mistakes DataFrame for comparison with actual charges
df_ms['predict'] = y_predict

In [16]:
# Displaying the mistakes DataFrame to review the actual charges ('y') alongside the predicted charges ('predict')
df_ms

,y,predict
0,16884.92400,25197.531061
1,1725.55230,3826.781929
2,4449.46200,6987.535290
3,21984.47061,3813.396616
4,3866.85520,5637.967419
...,...,...
1333,10600.54830,12243.889908
1334,2205.98080,3454.337913
1335,1629.83350,4458.339897
1336,2007.94500,1352.459258


In [17]:
# Calculating the error by subtracting predicted values from actual values
# This creates a new column in the mistakes DataFrame that shows the difference (error) for each prediction
df_ms['error'] = y - y_predict

In [18]:
# Displaying the first 10 rows of the mistakes DataFrame to review the actual charges, predicted charges, and errors
df_ms.head(10)

,y,predict,error
0,16884.92400,25197.531061,-8312.607061
1,1725.55230,3826.781929,-2101.229629
2,4449.46200,6987.535290,-2538.073290
3,21984.47061,3813.396616,18171.073994
4,3866.85520,5637.967419,-1771.112219
5,3756.62160,4053.925821,-297.304221
6,8240.58960,10929.452842,-2688.863242
7,7281.50560,7832.988481,-551.482881
8,6406.41070,8464.318765,-2057.908065
9,28923.13692,11827.117516,17096.019404


In [19]:
# Calculating the squared error by squaring the values in the 'error' column
# This creates a new column in the mistakes DataFrame to quantify the error for each prediction
df_ms['squared_error'] = df_ms['error'] ** 2

In [20]:
# Calculating the absolute error by taking the absolute values of the 'error' column
# This creates a new column in the mistakes DataFrame to represent the magnitude of the error without regard to direction
df_ms['abs_error'] = np.abs(df_ms['error'])

In [21]:
# Calculating the percentage error by dividing the absolute error by the actual values (y)
# This creates a new column in the mistakes DataFrame to express the error as a percentage of the actual charges
df_ms['percent_error'] = np.abs((y - y_predict) / y)

In [22]:
# Displaying the first 10 rows of the mistakes DataFrame to review the actual charges, predicted charges, errors, squared errors, absolute errors, and percentage errors
df_ms.head(10)

,y,predict,error,squared_error,abs_error,percent_error
0,16884.92400,25197.531061,-8312.607061,6.909944e+07,8312.607061,0.492309
1,1725.55230,3826.781929,-2101.229629,4.415166e+06,2101.229629,1.217714
2,4449.46200,6987.535290,-2538.073290,6.441816e+06,2538.073290,0.570423
3,21984.47061,3813.396616,18171.073994,3.301879e+08,18171.073994,0.826541
4,3866.85520,5637.967419,-1771.112219,3.136838e+06,1771.112219,0.458024
5,3756.62160,4053.925821,-297.304221,8.838980e+04,297.304221,0.079141
6,8240.58960,10929.452842,-2688.863242,7.229986e+06,2688.863242,0.326295
7,7281.50560,7832.988481,-551.482881,3.041334e+05,551.482881,0.075737
8,6406.41070,8464.318765,-2057.908065,4.234986e+06,2057.908065,0.321226
9,28923.13692,11827.117516,17096.019404,2.922739e+08,17096.019404,0.591085


In [23]:
df_ms.mean()

y                1.327042e+04
predict          1.331434e+04
error           -4.391368e+01
squared_error    3.654043e+07
abs_error        4.202818e+03
percent_error    4.314120e-01
dtype: float64

In [24]:
# Calculating the Mean Squared Error (MSE) between the actual values (y) and the predicted values (y_predict)
# This metric quantifies the average squared difference between predicted and actual values
mse = mean_squared_error(y, y_predict)

In [25]:
# Calculating the Mean Absolute Error (MAE) between the actual values (y) and the predicted values (y_predict)
# This metric represents the average absolute difference between predicted and actual values
mae = mean_absolute_error(y, y_predict)

In [26]:
# Calculating the Mean Absolute Percentage Error (MAPE) between the actual values (y) and the predicted values (y_predict)
# This metric expresses the average absolute error as a percentage of the actual values, providing insight into relative accuracy
mape = mean_absolute_percentage_error(y, y_predict)

In [27]:
# Creating a summary DataFrame to present all errors together
error_summary = pd.DataFrame({
    'Metric': ['Mean Squared Error', 'Mean Absolute Error', 'Mean Absolute Percentage Error'],
    'Value': [mse, mae, mape]
})
# Displaying the error summary DataFrame to review all calculated error metrics
error_summary

,Metric,Value
0,Mean Squared Error,3.654043e+07
1,Mean Absolute Error,4.202818e+03
2,Mean Absolute Percentage Error,4.314120e-01


 Purpose of Simplifying Error Values

We simplify the error values in the `error_summary` DataFrame to improve readability and interpretation. By rounding the numbers to a fixed number of decimal places, we avoid scientific notation (e.g., `1.23e+05`) and present the results in a more user-friendly format. This enhances clarity, making it easier to compare and understand the model's performance metrics.

Rounding the error values to a fixed number of decimal places (e.g., 2 decimal places)
code -> error_summary['Value'] = error_summary['Value'].apply(lambda x: round(x, 2))  # Rounding to 2 decimal places

Displaying the updated error summary DataFrame with simplified values
code -> error_summary